In [82]:
import sys
import os
sys.path.append("..")
from globals import ROOT_DIR
import configparser
import csv
import pandas as pd
import numpy as np 
import tweepy
config = configparser.ConfigParser()
config.read('../config.ini')
consumer_key = config['DEFAULT']['TWITTER_CONSUMER_KEY']
consumer_secret_key = config['DEFAULT']['TWITTER_CONSUMER_SECRET_KEY']
access_token = config['DEFAULT']['TWITTER_ACCESS_TOKEN']
access_token_secret = config['DEFAULT']['TWITTER_ACCESS_TOKEN_SECRET']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)


In [83]:
data = np.load(os.path.join(ROOT_DIR, 'data/founta_data.npy'))
data = data[()]


In [130]:
label_data = pd.read_csv(os.path.join(ROOT_DIR,'data/founta_data.csv'), header='infer', index_col=0, squeeze=True).to_dict()

In [107]:
replies = np.load(os.path.join(ROOT_DIR, 'data/reply_data.npy'))
replies = replies[()]
print(list(replies.keys())[0])

848608337696616448


In [168]:
word_cloud = [value['text'].lower() for key, value in data.items() if label_data[value['id']] == 'spam']

In [169]:
text = ' '.join(word_cloud)
text = text.replace('amp', '')
text

'@thorsen1nk that is a fucking movie! https://t.co/ywg7izywkk i can\'t wait til i get cleared that my cut is healed that way i can start working on losing this ugly ass stomach javien gave me 😩 click here to watch the movie:  https://t.co/tz5cygzmmv\nbad ass bootys https://t.co/fk7u9jopze just watched @chicken_film woahhhhh break my fucking heart! rip fiona  @freddie_machin rt @amyjaimexl: stupid girl with the ugly ass hair https://t.co/ypbgeql9xn rt @rodnishalashawn: facts, mad hoes link up! https://t.co/folefw9yfv rt @yg_winner: [!] winner #reallyreally at #1 and #fool at #8 on melon (9pm kst)! 🍈 👏🏻💪🏻💙 https://t.co/f8dcmd80ri @musclybabies @danthedazzler @kidlykos @obeybrookes @attackwrestling something something idiot rt @mnet101boys: [press] 170403 kuan lin and seon ho #라이관린 #유선호 @ produce 101 season 2 press conference #프로듀스101 #produce101 https://t.co/i… rt @onlylookatmino: [!!] #winner trending on melon search🍈\n#fatenumberfor\n#reallyreally\n💚💜💛💙 https://t.co/obabwubvaq rt @winn

In [148]:
count = 0
CHOICE = 20
for key, value in data.items():
    if count > CHOICE:
        break 
    reply_status_id = value['in_reply_to_status_id_str']
    if reply_status_id and reply_status_id in replies and label_data[value['id']] == 'spam':
        count += 1
        print("\n")
        print(replies[reply_status_id])
        print(value['text'])
     



@Thai_Daishiki Start this at 1:48 for just a taste of the madness: https://t.co/O8gn0zLLmx
@thorsen1nk THAT IS A FUCKING MOVIE! https://t.co/ywG7IZywKk


I FUCKING MISSED THE SHOES FUCK
just fucking kidding! scored https://t.co/VzAAxQtRDZ


@Khanoisseur @jack Yeah....it's stopping the Dems and Inds from making its points about T-Rump.
@Glorygirlone @Khanoisseur @jack Wait .. can anyone replying to see this tweet?  Is that why no one can see my twee… https://t.co/dGo3Sq2MXo


70% of Hydro One still belong to the people of Ontario. Stop the sale. @Kathleen_Wynne @GlennThibeault… https://t.co/Kmc1O4Wafm
@KeepHydroPublic @Kathleen_Wynne @GlennThibeault Is it really public the way I see it if we all believe that then s… https://t.co/dHiL5heQVj


Ambassador John Bolton: Unmasker Susan Rice "Has Serious Legal Problems" (VIDEO) https://t.co/Pe1pFdlvC8
@mitchellvii Convene a Grand Jury, and let us get started, nothing can happen without one. https://t.co/g8ZLEh9OkB


#amexgift
@qunqunspg1 Thx

In [18]:
isolated_count = 0 
count = 0
context_tweets = {}
for key, value in data.items():
    tweet = value['id']
    count += 1
    print(count)
    original_tweet = value['in_reply_to_status_id']
    if original_tweet is None:
        context_tweets[value['id']] = fetch_replies(value['user']['screen_name'], tweet) #list of lists
        if len(context_tweets[value['id']]) == 0:
            isolated_count +=1 
print(isolated_count, len(data.keys()))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 

In [49]:
for key, value in context_tweets.items():
    if len(value) != 0:
        print(len(value))

In [48]:
def fetch_replies(screen_name, tweet_status_id):
    context_tweets = {}
    res = api.search(q='to:{}'.format(screen_name))
    for item in res:
        in_reply_to = item._json['in_reply_to_status_id']
        if not in_reply_to:
            continue
        if int(in_reply_to) == int(tweet_status_id):
            context_tweets[item._json['id']] = item._json 
    return context_tweets
item = fetch_replies('goodfellow_ian', 1149429779814203392)
print(len(item)) # should be 83 

15


In [34]:
res = api.search(q='1149429779814203392')
res

[Status(_api=<tweepy.api.API object at 0x153605828>, _json={'created_at': 'Sat Jul 13 11:01:20 +0000 2019', 'id': 1149997233041018885, 'id_str': '1149997233041018885', 'text': 'Good man! https://t.co/Pe88fGN6pR', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/Pe88fGN6pR', 'expanded_url': 'https://twitter.com/goodfellow_ian/status/1149429779814203392', 'display_url': 'twitter.com/goodfellow_ian…', 'indices': [10, 33]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 158372108, 'id_str': '158372108', 'name': "'Tahir", 'screen_name': 'EnigmaticMe_', 'location': 'Literary Oblivion!', 'description': 'El - Observador', 'url': 'https://

In [44]:
for item in res:
    print(item._json['user']['screen_name'])

EnigmaticMe_
nerdCyberArtist
databytz
BedabrataChoud1
M_BUEHLER
captain__pool
cbyad1
doomie
skynetcap


In [ ]:
i = 0
for key, value in data.items():
    print(i)
    i+=1
    context_tweets = fetch_replies(value['user']['screen_name'], value['id'])
#     context_tweets = fetch_replies('goodfellow_ian', '1149429779814203392')
    if len(context_tweets) > 0:
        print(value['id'])
        break

In [ ]:
status_ids = list(data.keys())
status_ids[0]

In [ ]:
def fetch_founta_data():
    founta_data = np.load(os.path.join(ROOT_DIR, 'data/founta_data.npy'))
    founta_data = founta_data[()]
    return founta_data

In [ ]:
# start_ptr = 0
# end_ptr = start_ptr + 100 
# while(start_ptr <= len(status_ids)):
#     print("Start ptr is at {}".format(start_ptr))
#     tweets = api.statuses_lookup(status_ids[start_ptr:end_ptr])
    
#     # load and save results each time 
#     if os.path.exists(os.path.join(ROOT_DIR, 'data/founta_data.npy')):
#         fetch_founta_data()
#     else: 
#         founta_data = {}
#     for i, tweet in enumerate(tweets):
#         tweet = tweet._json
#         founta_data[tweet['id_str']] = tweet
#     np.save(os.path.join(ROOT_DIR, 'data/founta_data.npy'), founta_data)
    
#     start_ptr += 100 
#     end_ptr += 100

In [ ]:
founta_data = fetch_founta_data()

In [ ]:
len(founta_data.keys())

In [ ]:
64149/99799

In [ ]:
list(founta_data.keys())[0]

In [ ]:
founta_data['848337741813358592']

In [ ]:
labels = [data[int(key)] for key in founta_data.keys() if int(key) in data and data[int(key)] != 'spam']
len(labels)

In [ ]:
for class_name in ['abusive', 'hateful', 'normal']:
    print("{} {:0.2f}%".format(class_name, (labels.count(class_name) / len(labels)) * 100))
len(labels)

In [ ]:
my_labels_file = os.path.join(ROOT_DIR, 'data/labels.csv')
my_labels = pd.read_csv(data_file, header='infer', index_col=0, squeeze=True).to_dict()
list(my_labels.keys())[0]

In [ ]:
import json
my_data_file = os.path.join(ROOT_DIR, 'data/80k_tweets.json')
all_my_labels = []
with open(my_data_file, 'r') as f:
    for line in f.readlines():
        obj = json.loads(line)
        status_id = int(obj['id_str'])
        if status_id not in my_labels:
            print("err")
        else:
            if my_labels[status_id] == 'spam':
                continue
            all_my_labels.append(my_labels[status_id])

In [ ]:
for class_name in ['abusive', 'hateful', 'spam', 'normal']:
    print("{} {:0.2f}%".format(class_name, (all_my_labels.count(class_name) / len(all_my_labels)) * 100))

In [ ]:
print(len(all_my_labels))

In [ ]:
## Determine that this new data is better 
from collections import Counter

retweet_count_hateful = []
retweet_count_normal = []
statuses = []
user_ids = []
for key, value in founta_data.items():
    label = data[value['id']]
    if label == 'hateful':
        retweet_count_hateful.append(1 if value['in_reply_to_status_id'] else 0)
    else:
        retweet_count_normal.append(1 if value['in_reply_to_status_id'] else 0)


In [ ]:
np.mean(retweet_count_hateful)

In [ ]:
np.mean(retweet_count_normal)

In [ ]:
(25 - 19) / 19

In [ ]:
(13 - 3)/ 13

In [ ]:
245 * 1.142

In [51]:
from sklearn.preprocessing import OneHotEncoder

In [53]:
favorites = [value['favorite_count'] for key, value in data.items()]

In [54]:
retweets = [value['retweet_count'] for key, value in data.items()]

In [77]:
max(retweets), np.mean(retweets)

(154565, 247.3795226737751)

In [78]:
max(favorites), np.mean(favorites)

(27741, 3.689644421581007)

In [69]:
np.log(27741) > np.log()

True

In [71]:
np.log(50)

3.912023005428146

In [76]:
index = np.log(40) * 10
print(index)

36.88879454113936


NameError: name 'x_train' is not defined